In [ ]:
import os
import random
from PIL import Image
import zipfile


In [ ]:
with zipfile.ZipFile("Project.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
with zipfile.ZipFile("Trees.zip", 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
image_folder = "Project/Merged_Dataset/images"
label_folder = "Project/Merged_Dataset/labels"
tree_folder = "Trees"
output_folder = "Occlusion_Output"

# Make sure output directory exists
os.makedirs(output_folder, exist_ok=True)

In [ ]:
import os
import random
from PIL import Image

# === CONFIGURATION ===
image_folder = "Project/Merged_Dataset/images"
label_folder = "Project/Merged_Dataset/labels"
tree_folder = "Trees"
output_folder = "output3"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Load all tree PNGs
tree_paths = [os.path.join(tree_folder, f) for f in os.listdir(tree_folder) if f.endswith(".png")]
trees = [Image.open(p).convert("RGBA") for p in tree_paths]

if not trees:
    raise Exception("No tree PNGs found!")

print(f"Loaded {len(trees)} tree images.")

# Handle resampling across Pillow versions
try:
    resample = Image.Resampling.LANCZOS
except AttributeError:
    resample = Image.ANTIALIAS

# Process each image
image_files = [f for f in os.listdir(image_folder) if f.lower().endswith((".jpg", ".jpeg", ".png"))]

for image_name in image_files:
    image_path = os.path.join(image_folder, image_name)
    label_path = os.path.join(label_folder, os.path.splitext(image_name)[0] + ".txt")

    if not os.path.exists(label_path):
        print(f"Skipping {image_name}: no annotation file")
        continue

    # Open image
    base_img = Image.open(image_path).convert("RGBA")
    img_width, img_height = base_img.size

    # Read annotations
    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()
        if len(parts) != 5:
            continue

        class_id, x_center, y_center, width, height = map(float, parts)

        # Convert normalized YOLO coordinates to pixel values
        box_w = int(width * img_width)
        box_h = int(height * img_height)
        x1 = int((x_center - width / 2) * img_width)
        y1 = int((y_center - height / 2) * img_height)

        # Load and scale tree
        base_tree = random.choice(trees).copy()
        scale_factor = random.uniform(1.5, 3.5)

        tree_w, tree_h = base_tree.size
        aspect_ratio = tree_w / tree_h
        max_dim = int(max(box_w, box_h) * scale_factor)

        if aspect_ratio > 1:
            new_w = max_dim
            new_h = int(max_dim / aspect_ratio)
        else:
            new_h = max_dim
            new_w = int(max_dim * aspect_ratio)

        tree_img = base_tree.resize((new_w, new_h), resample)
        tree_img = tree_img.rotate(random.choice([0, 90, 180, 270]), expand=True)

        # CENTERED placement over car
        paste_x = x1 + (box_w - new_w) // 2
        paste_y = y1 + (box_h - new_h) // 2

        # Clip to image bounds
        paste_x = max(0, min(paste_x, img_width - new_w))
        paste_y = max(0, min(paste_y, img_height - new_h))

        # Paste with transparency
        base_img.paste(tree_img, (paste_x, paste_y), tree_img)

    # Save the final image
    out_path = os.path.join(output_folder, image_name)
    base_img.convert("RGB").save(out_path)
    print(f"Saved: {out_path}")


Loaded 11 tree images.
✅ Saved: output3/2 (34)_1650425225_aug0.jpg
✅ Saved: output3/1 (19)_1649990929.jpg
✅ Saved: output3/2013-02-22_17_10_11_aug0.jpg
✅ Saved: output3/steptodown.com393941_aug0.jpg
✅ Saved: output3/21_aug0.jpg
✅ Saved: output3/steptodown.com694149_aug2.jpg
✅ Saved: output3/steptodown.com782779_aug0.jpg
✅ Saved: output3/20_aug1.jpg
✅ Saved: output3/4 (27)_1649991515_aug1.jpg
✅ Saved: output3/steptodown.com106658_aug0.jpg
✅ Saved: output3/2 (34)_1650425225_aug1.jpg
✅ Saved: output3/2 (50)_1650423534_aug1.jpg
✅ Saved: output3/4 (12)_1649859974_aug0.jpg
✅ Saved: output3/1 (34)_1650425207_aug1.jpg
✅ Saved: output3/2_aug2.jpg
✅ Saved: output3/4 (19)_1649991443_aug0.jpg
✅ Saved: output3/steptodown.com777376.jpg
✅ Saved: output3/steptodown.com828042.jpg
✅ Saved: output3/steptodown.com893372_aug1.jpg
✅ Saved: output3/5 (24)_1649991661_aug2.jpg
✅ Saved: output3/4_aug1.jpg
✅ Saved: output3/4 (12)_1649859974_aug1.jpg
✅ Saved: output3/3 (49)_1650423576_aug2.jpg
✅ Saved: output3/3 